<a href="https://colab.research.google.com/github/exoticTraderSisyphus/International_Parity_Conditions/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np
import requests
import json
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_PATH = "/content/drive/MyDrive/International_Parity_Conditions/Data_Sources"

In [4]:
# Base URL used in all the API calls
BASE_URL='http://api.worldbank.org/v2/'

# List of indicators according to the features defined above
INDICATOR_CODES=[
  "FP.CPI.TOTL.ZG",
  "FP.CPI.TOTL",
  "BN.CAB.XOKA.CD", 
  "BN.GSR.GNFS.CD", 
  "BX.GSR.GNFS.CD", 
  "BM.GSR.GNFS.CD",
  "BN.TRF.KOGT.CD",
  "BX.KLT.DINV.CD.WD",
  "BM.KLT.DINV.CD.WD",
  "BN.KLT.PTXL.CD",
  "GC.DOD.TOTL.CN",
  "NY.GDP.MKTP.CD",
  "NY.GDP.PCAP.CN",
  "FM.LBL.BMNY.CN",
  "FM.AST.NFRG.CN"]

COUNTRY_LIST=['USA', 'China', 'Japan', 'Great Britain']

# mapping of feature codes to more meaningful names
featureMap={
  "FP.CPI.TOTL.ZG": "Annual Inflation Rate",
  "FP.CPI.TOTL": "CPI",
  "BN.CAB.XOKA.CD": "Current Account Balance",
  "BN.GSR.GNFS.CD": "Net trade in goods and services",
  "BX.GSR.GNFS.CD": "Exports in goods and services",
  "BM.GSR.GNFS.CD": "Imports in goods and services",
  "BN.TRF.KOGT.CD": "Net Capital Account",
  "BX.KLT.DINV.CD.WD": "Foreign Direct Investment, net inflows",
  "BM.KLT.DINV.CD.WD": "Foreign Direct Investments, net ouflows",
  "BN.KLT.PTXL.CD": "Net Portfolio Investment",
  "GC.DOD.TOTL.CN": "Central Government Debt",
  "NY.GDP.MKTP.CD": "GDP",
  "NY.GDP.PCAP.CN": "GDP per capita",
  "FM.LBL.BMNY.CN": "Broad Money",
  "FM.AST.NFRG.CN": "Net Foreign Assets"
}

# Mapping of country codes to their actual names
countryMap={
    "US": "USA",
    "CN": "China",
    "JP": "Japan",
    "GB": "Great Britain",
}

# constant parameters used in sending the request.
params = dict()
# to ensure we receive a JSON response
params['format']='json'
# The data we fetch is for 59 years.
# Hence we change the default page size of 50 to 100 to ensure we need only one API call per feature.
params['per_page']='100'
# Range of years for which the data is needed
start, end = 1960, 2021
params['date']=f'{start}:{end}'


In [5]:
params

{'format': 'json', 'per_page': '100', 'date': '1960:2021'}

In [6]:
# Function to get JSON data from the endpoint
def loadJSONData(country_code): 
    dataList=[]
    
    # iterate over each indicator code specified in the contant INDICATOR_CODES defined above
    for indicator in INDICATOR_CODES: 
        
        # form the URL in the desired format
        # E.g: http://api.worldbank.org/v2/countries/us/indicators/SP.POP.TOTL?format=json&per_page=200&date=1960:2018
        url=BASE_URL+'countries/'+country_code.lower()+'/indicators/'+indicator
        
        # send the request using the resquests module
        response = requests.get(url, params=params)
        
        # validate the response status code
        # The API returns a status_code 200 even for error messages,
        # however, the response body contains a field called "message" that includes the details of the error
        # check if message is not present in the response
        if response.status_code == 200 and ("message" not in response.json()[0].keys()):
            # print("Successfully got data for: " + str(featureMap[indicator]))
            
            # list of values for one feature
            indicatorVals=[]
            
            # the response is an array containing two arrays - [[{page: 1, ...}], [{year: 2018, SP.POP.TOTL: 123455}, ...]]
            # hence we check if the length of the response is >1
            if len(response.json()) > 1:
                
                # if yes, iterate over each object in the response
                # each object gives one single value for each year
                for obj in response.json()[1]:
                    
                    # check for empty values
                    if obj['value'] == "" or obj['value'] == None:
                        indicatorVals.append(None)
                    else:
                    # if a value is present, add it to the list of indicator values
                        indicatorVals.append(float(obj['value']))
                dataList.append(indicatorVals)
        else:
            # print an error message if the API call failed
            print("Error in Loading the data. Status Code: " + str(response.status_code))
            
    # Once all the features have been obtained, add the values for the "Year"
    # The API returns the indicator values from the most recent year. Hence, we create a list of years in reverse order
    dataList.append([year for year in range(end, start-1, -1)])
    # return the list of lists of feature values [[val1,val2,val3...], [val1,val2,val3...], [val1,val2,val3...], ...]
    return dataList

#----------------------------------------------------------------------------------------------------

# function to invokde the loadJSONData function and form the final DataFrame for each country
def getCountrywiseDF(country_code):
    
    # The resulting dataframe needs to have meaningful column names
    # hence we create a list of column names from the map defined above
    col_list=list(featureMap.values())
    # append the year column name
    col_list.append('Year')
    
    print("------------------Loading data for: "+countryMap[country_code]+"-----------------------")
    
    # for the given country call the loadJSONData function and fetch the data from the API
    dataList=loadJSONData(country_code)
    
    # transform the list of lists of features into a DataFrame
    # np.column_stack is used to add each list as a column 
    df=pd.DataFrame(np.column_stack(dataList), columns=col_list)
    
    # add the country column by extracting the country name from the map using the country code
    df['Country'] = countryMap[country_code]
    
    # display the resulting dataframe
    display(df.head())
    
    # return the formed dataframe for the given country
    return df

In [7]:
US_df=getCountrywiseDF('US')
CN_df=getCountrywiseDF('CN')
JP_df=getCountrywiseDF('JP')
GB_df=getCountrywiseDF('GB')

------------------Loading data for: USA-----------------------


,Annual Inflation Rate,CPI,Current Account Balance,Net trade in goods and services,Exports in goods and services,Imports in goods and services,Net Capital Account,"Foreign Direct Investment, net inflows","Foreign Direct Investments, net ouflows",Net Portfolio Investment,Central Government Debt,GDP,GDP per capita,Broad Money,Net Foreign Assets,Year,Country
0,4.697859,124.266414,-846354000000.0,-845050000000.0,2556638000000.0,3401688000000.0,-2475000000.0,448324000000.0,421750000000.0,42982000000.0,None,23315080560000.0,70248.629,None,None,2021,USA
1,1.233584,118.690502,-619702000000.0,-653990000000.0,2158651000000.0,2812641000000.0,-5532000000.0,148912000000.0,271797000000.0,-540198000000.0,26618437000000.0,21060473613000.0,63530.633484,23301047844003.398438,307553379930.052979,2020,USA
2,1.81221,117.244195,-445955000000.0,-559675000000.0,2546280000000.0,3105955000000.0,-6455000000.0,314744000000.0,105677000000.0,-244920000000.0,21573955000000.0,21380976119000.0,65120.394663,19881523881838.0,256812153769.386993,2019,USA
3,2.442583,115.157303,-439851000000.0,-578600000000.0,2542459000000.0,3121059000000.0,-4261000000.0,214715000000.0,-130720000000.0,78785000000.0,20358441000000.0,20533057312000.0,62823.309438,18342259948911.800781,-74874080291.652802,2018,USA
4,2.13011,112.411557,-361018000000.0,-510339000000.0,2394477000000.0,2904816000000.0,12395000000.0,380823000000.0,409413000000.0,-250083000000.0,19043953000000.0,19477336549000.0,59907.754261,17631631156299.0,-219901833247.987,2017,USA


------------------Loading data for: China-----------------------


,Annual Inflation Rate,CPI,Current Account Balance,Net trade in goods and services,Exports in goods and services,Imports in goods and services,Net Capital Account,"Foreign Direct Investment, net inflows","Foreign Direct Investments, net ouflows",Net Portfolio Investment,Central Government Debt,GDP,GDP per capita,Broad Money,Net Foreign Assets,Year,Country
0,0.981015,129.366217,317300990626.927979,462807887089.710022,3554296099846.049805,3091488212756.339844,91500853.54,333979029130.320007,128037380367.690002,-50961837070.82,None,17734062645371.398438,80975.792291,None,None,2021,China
1,2.419422,128.109444,248835643936.040009,358572608160.02002,2738897986700.97998,2380325378540.959961,-75764996.64,253095616058.579987,153720808338.109985,-95539019942.699997,None,14687673892882.0,71828.148253,214766371891765.0,26886430520561.898438,2020,China
2,2.899234,125.083154,102909876008.388,131844297312.037003,2630999236934.950195,2499154939622.910156,-327297478.87,187169822364.755005,136910000000.0,-57947645385.638,None,14279937500606.5,70077.69179,195218087092332.0,26392404962887.5,2019,China
3,2.07479,121.558878,24130909184.498501,87905148451.873596,2651009602168.899902,2563104453717.029785,-568607666.62,235365050036.341003,143026576683.860992,-106873525546.720993,None,13894817549374.199219,65533.742698,179292800240995.0,25506987279521.601562,2018,China
4,1.593136,119.08805,188676126121.161987,217009899933.946991,2429277356205.390137,2212267456271.439941,-91496073.44,166083755721.648987,138292767802.139008,-29497793098.263302,None,12310409370892.800781,59592.25109,167577363382557.0,25263930561710.0,2017,China


------------------Loading data for: Japan-----------------------


,Annual Inflation Rate,CPI,Current Account Balance,Net trade in goods and services,Exports in goods and services,Imports in goods and services,Net Capital Account,"Foreign Direct Investment, net inflows","Foreign Direct Investments, net ouflows",Net Portfolio Investment,Central Government Debt,GDP,GDP per capita,Broad Money,Net Foreign Assets,Year,Country
0,-0.233353,105.211812,157743004558.451996,-22513457717.494099,919158028080.979004,941671485798.470947,-3824968251.19206,33092988100.898399,210778617284.787994,-196648731488.277008,None,4940877780755.330078,4314734.457575,None,None,2021,Japan
1,-0.024996,105.457901,147948273293.911987,-7597675867.01364,794291076749.503052,801888752616.516968,-1932522234.64785,60725202927.858704,146178021894.320007,35926915439.337196,1163916500000000.0,5040107754084.110352,4262245.665724,1513826500000000.0,92681600000000.0,2020,Japan
2,0.468776,105.484268,176609997619.696014,-8615189108.462191,904632403731.96106,913247592840.422974,-3808661034.19253,39960544340.014099,258284325075.721985,86642953831.485596,1104706200000000.0,5123318151510.620117,4410313.267474,1409903600000000.0,79624700000000.0,2019,Japan
3,0.989095,104.99209,177268535276.423004,1405006849.05243,929827552588.353027,928422545739.300049,-1922509970.18999,25289367857.851799,160218306051.632996,92911832022.275696,1098381600000000.0,5037835383110.969727,4386794.52098,1381142200000000.0,80941800000000.0,2018,Japan
4,0.4842,103.96379,203168727082.394989,37636176718.934196,875513551319.491943,837877374600.557983,-2498993250.54329,18802251208.106899,173750236904.079987,-49213900461.9487,1070162900000000.0,4930837369151.419922,4355865.86019,1349607100000000.0,83451500000000.0,2017,Japan


------------------Loading data for: Great Britain-----------------------


,Annual Inflation Rate,CPI,Current Account Balance,Net trade in goods and services,Exports in goods and services,Imports in goods and services,Net Capital Account,"Foreign Direct Investment, net inflows","Foreign Direct Investments, net ouflows",Net Portfolio Investment,Central Government Debt,GDP,GDP per capita,Broad Money,Net Foreign Assets,Year,Country
0,2.518371,123.848715,-62941408374.711098,-24233324660.438202,874485940890.021973,898719265550.459961,-3562713673.49336,5922252759.15616,160931982890.44101,-322831983365.591003,None,3131377762925.950195,33815.996178,None,None,2021,Great Britain
1,0.989487,120.806362,-87814101834.237198,8295267014.14478,791733492696.873047,783438225682.728027,-3924036037.06856,132476034789.014999,-4444825595.10756,31995557917.767399,4121984000000.0,2704609160088.149902,31448.457835,3447481000000.0,217637000000.0,2020,Great Britain
2,1.738105,119.622711,-80778655725.834198,-46207199363.950104,893141892842.457031,939349092206.406982,-1689172144.8505,19790761929.127701,-21498303884.121601,34756469387.769897,3581810000000.0,2857057847953.02002,33489.991154,3142460000000.0,281300000000.0,2019,Great Britain
3,2.29284,117.579064,-116991662017.686005,-43882686819.431602,899005981882.619019,942888668702.051025,-3585311002.35494,-25055440306.907299,-28632134257.130402,-353571263790.36499,3404472000000.0,2878152147315.819824,32461.6135,3185921000000.0,396481000000.0,2018,Great Britain
4,2.557756,114.943592,-96879427831.574799,-38943461352.965797,824360223649.562012,863303685002.526978,-2321009114.69037,125358809934.121002,172043885461.575989,-92026807624.250504,3358710000000.0,2683399006715.790039,31562.882429,3025571000000.0,404609000000.0,2017,Great Britain


In [8]:
US_df.to_csv("/content/drive/MyDrive/Colab Notebooks/International_Parity_Conditions/Data_Cache/USdata.csv", index = False)
CN_df.to_csv("/content/drive/MyDrive/Colab Notebooks/International_Parity_Conditions/Data_Cache/CNdata.csv", index = False)
JP_df.to_csv("/content/drive/MyDrive/Colab Notebooks/International_Parity_Conditions/Data_Cache/JPdata.csv", index = False)
GB_df.to_csv("/content/drive/MyDrive/Colab Notebooks/International_Parity_Conditions/Data_Cache/GBdata.csv", index = False)


In [ ]:
response = requests.get("https://stats.oecd.org/sdmx-json/data/DP_LIVE/.TRUSTGOV.TOT.PC.A")

In [ ]:
response.json()

{'header': {'id': '207ee2fd-869c-4aba-a236-b12aa1f498fc',
  'test': False,
  'prepared': '2023-04-01T18:19:45.6016679Z',
  'sender': {'id': 'OECD',
   'name': 'Organisation for Economic Co-operation and Development'},
  'links': [{'href': 'https://stats.oecd.org:443/sdmx-json/data/DP_LIVE/.TRUSTGOV.TOT.PC.A',
    'rel': 'request'}]},
 'dataSets': [{'action': 'Information',
   'series': {'0:0:0:0:0': {'attributes': [0],
     'observations': {'0': [53.206332837474, None],
      '1': [52.6932810746771, None],
      '2': [64.4675424053232, None],
      '3': [60.9752918764891, None],
      '4': [53.0786719332192, None],
      '5': [42.0419178918908, None],
      '6': [45.5871468172797, None],
      '7': [46.4676233517878, None],
      '8': [47.855683708198, None],
      '9': [45.2731750058896, None],
      '10': [45.3406962891371, None],
      '11': [46.8836874513729, None],
      '12': [46.8700014928596, None],
      '13': [44.6422249484244, None],
      '14': [51.9226387269998, None],
   

In [ ]:
from pathlib import Path

Parsing the Aution Results

In [ ]:
dfs = pd.ExcelFile("/content/drive/MyDrive/Colab Notebooks/International_Parity_Conditions/To Do List/Task 1/Auction_Results_for_T-bills (Japan).xls")
dfs.sheet_names

['FY2022',
 'FY2021',
 'FY2020',
 'FY2019',
 'FY2018',
 'FY2017 ',
 'FY2016',
 'FY2015',
 'FY2014 ',
 'FY2013',
 'FY2012',
 'FY2011',
 'FY2010',
 'FY2009',
 'FY2008']

In [ ]:
i = 0
df = pd.read_excel(dfs, dfs.sheet_names[i])

In [ ]:
## formatting the df so it's normal
df.rename(columns=df.iloc[1], inplace = True)
df.drop(df.index[[0,1]], inplace = True)
df

,Issue Number,Maturity,Auction Date,Issue Date,Maturity Date,[A+B]\n(billion yen),Amounts of Competitive Bids (billion yen),Amounts of Bids Accepted (billion yen)\n[A]\n,Average Price,Yield at the Average Price(%),Lowest Accepted Price,Yield at the Lowest Accepted Price(%),Non-price -competitive Auction Ⅰ (billion yen)\n[B]
2,1070,3-month,2022-04-01 00:00:00,2022-04-04 00:00:00,2022-07-11 00:00:00,5699.91,16534.7,4620.41,100.0263,-0.0979,100.025,-0.093,1079.5
3,1071,6-month,2022-04-07 00:00:00,2022-04-11 00:00:00,2022-10-11 00:00:00,3399.97,13729.1,2756.27,100.049,-0.0976,100.048,-0.0956,643.7
4,1072,3-month,2022-04-08 00:00:00,2022-04-11 00:00:00,2022-07-19 00:00:00,5899.96,20067.05,4782.16,100.0332,-0.1223,100.032,-0.1179,1117.8
5,1073,3-month,2022-04-15 00:00:00,2022-04-18 00:00:00,2022-07-25 00:00:00,5899.94,11653.95,4782.14,100.0303,-0.1128,100.027,-0.1005,1117.8
6,1074,1-year,2022-04-19 00:00:00,2022-04-20 00:00:00,2023-04-20 00:00:00,3499.98,8742.4,2837.18,100.079,-0.0789,100.066,-0.0659,662.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1137,6-month,2023-02-09 00:00:00,2023-02-10 00:00:00,2023-08-10 00:00:00,4499.96,15397.8,3691.46,100.066,-0.133,100.057,-0.1148,808.5
70,1138,3-month,2023-02-10 00:00:00,2023-02-13 00:00:00,2023-05-22 00:00:00,6299.93,15445.1,5136.53,100.0443,-0.1649,100.0415,-0.1545,1163.4
71,1139,1-year,2023-02-16 00:00:00,2023-02-20 00:00:00,2024-02-20 00:00:00,3499.98,10868.1,2847.68,100.096,-0.0959,100.086,-0.0859,652.3
72,1140,3-month,2023-02-17 00:00:00,2023-02-20 00:00:00,2023-05-29 00:00:00,6299.94,16529.1,5125.14,100.041,-0.1526,100.0365,-0.1358,1174.8


In [ ]:
pvt = df.pivot_table(index=["Maturity", "Auction Date"], values="Yield at the Average Price(%)")
pvt

Yield at the Average Price(%)
Maturity Auction Date                               
1-year   2022-04-19                          -0.0789
         2022-05-19                          -0.0884
         2022-06-15                          -0.0949
         2022-07-19                          -0.1308
         2022-08-18                          -0.1311
...                                              ...
6-month  2022-10-06                          -0.1712
         2022-11-09                          -0.1148
         2022-12-08                          -0.1322
         2023-01-06                          -0.1007
         2023-02-09                          -0.1330

[72 rows x 1 columns]

In [ ]:
pvt.reset_index(drop = False, inplace= True)
pvt

,Maturity,Auction Date,Yield at the Average Price(%)
0,1-year,2022-04-19,-0.0789
1,1-year,2022-05-19,-0.0884
2,1-year,2022-06-15,-0.0949
3,1-year,2022-07-19,-0.1308
4,1-year,2022-08-18,-0.1311
...,...,...,...
67,6-month,2022-10-06,-0.1712
68,6-month,2022-11-09,-0.1148
69,6-month,2022-12-08,-0.1322
70,6-month,2023-01-06,-0.1007


In [ ]:
pvt.Maturity.unique()

array(['1-year', '3-month', '6-month'], dtype=object)

In [ ]:
three_months = pvt[pvt['Maturity']=="3-month"][["Auction Date", "Yield at the Average Price(%)"]]
three_months.head()

,Auction Date,Yield at the Average Price(%)
11,2022-04-01,-0.0979
12,2022-04-08,-0.1223
13,2022-04-15,-0.1128
14,2022-04-22,-0.1191
15,2022-05-06,-0.1343


In [ ]:
three_months["Auction Date"] = pd.DatetimeIndex(three_months["Auction Date"]) ## set as datetime index
three_months = three_months.set_index("Auction Date", drop = True)

## upsample the data, then time weight interpolation
three_months = three_months.resample('D').interpolate('time')

In [ ]:
three_months.head()

,Yield at the Average Price(%)
Auction Date,
2022-04-01,-0.097900
2022-04-02,-0.101386
2022-04-03,-0.104871
2022-04-04,-0.108357
2022-04-05,-0.111843
